In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sales=pd.read_csv('../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv',header=0)
sales.head()

<h2>Its time for some EDA</h2>

<b>Distribution of price</b>

In [ ]:
sales['price'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Value of price')
plt.ylabel('Frequency')

Most products are priced between 5 to 10 Euros. 

<b>Distribution of Retail price</b>

In [ ]:
sales['retail_price'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Retail price')
plt.ylabel('Frequency')

<b>Distribution of Units sold</b>

In [ ]:
sales['units_sold'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Units Sold')
plt.ylabel('Frequency')

<b>Distribution of Rating</b>

In [ ]:
sales['rating'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Rating')
plt.ylabel('Frequency')

<b>Distribution of Number of rating</b>

In [ ]:
sales['rating_count'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Rating Count')
plt.ylabel('Frequency')

<b>Distribution of number of Badges</b>

In [ ]:
sales['badges_count'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Number of Badges')
plt.ylabel('Frequency')

<b>Distribution of Shipping Cost</b>

In [ ]:
sales['shipping_option_price'].plot(kind='hist',figsize=(10,10))
plt.xlabel('Shipping Cost')
plt.ylabel('Frequency')

<b>Country wise distribution of total number of products</b>

In [ ]:
sales.groupby('origin_country')['product_id'].count().sort_values(ascending=False).plot(kind='bar',figsize=(10,10),title='Country wise products count')

In [ ]:
sales.info()

<p1><b>There are several columns that we won't need, for instance 'merchant_has_profile_picture','merchant_profile_picture', which can be removed</b></p1>

In [ ]:
to_be_removed=['merchant_title','merchant_has_profile_picture','merchant_profile_picture','product_url','product_picture','crawl_month','has_urgency_banner','urgency_text']

In [ ]:
sales.drop(to_be_removed,axis=1,inplace=True)
sales.info()

<p1><b>We have managed to remove 7 columns which might have nothing to contribute to our analysis.</b></p1><p2>That being said, lets articulate what needs to be achieved in this project:</p2>

<p3>Based on available data about products we would have to figure out those products that are potential hot cakes. For this let use K means clustering based approach. Lets classify the products into hot, warm, cold and very cold products based on their sales </p3>

In [ ]:
sales.describe()

<h2><b>What matters to users is the amount that can be saved by purchasing the product here. Hence lets add one more column</b></h2>

In [ ]:
sales['price_difference']=sales['retail_price']-sales['price']

In [ ]:
sales.columns

<p1><b>Lets see how many different currencies are supported in this platform</b></p1>

In [ ]:
sales['currency_buyer'].value_counts()

Lets see how various variables in this data set are correlated to each other


In [ ]:
to_be_used=['price','retail_price','uses_ad_boosts','rating','rating_count','rating_five_count','rating_four_count','rating_three_count','rating_two_count','rating_one_count','badges_count','badge_local_product','badge_product_quality','badge_fast_shipping','shipping_option_price','shipping_is_express','countries_shipped_to','merchant_rating_count','units_sold']

<b>Lets plot the correlation of various variables with the 'units sold' variable to see which are the influential ones</b>

In [ ]:
f=plt.figure(figsize=(25,25))
plt.matshow(sales[to_be_used].corr(),fignum=f.number)
plt.xticks(range(sales[to_be_used].shape[1]), sales[to_be_used].columns, fontsize=14, rotation=45)
plt.yticks(range(sales[to_be_used].shape[1]), sales[to_be_used].columns, fontsize=14)
cb=plt.colorbar()
cb.ax.tick_params(labelsize=14)

<b>Lets focus only on the factor that exert a descent amount of influence on the sales</b>

In [ ]:
l=[]

for i in to_be_used:
    if abs(sales['units_sold'].corr(sales[i])>=0.05):
        l.append(i)
l.remove('units_sold')
l

<p1><b>We have a seen the influence of numerical variables on the sales. Lets see the impact of the categorical variable on sales. People do get influenced by other factors like Counrty of origin, merchant details, etc</b></p1>

Lets see what are all the categorical variables present in this data set

In [ ]:
sales.describe(include='object')

In [ ]:
sales['shipping_option_name'].value_counts()

In [ ]:
sales['origin_country'].value_counts()

<p1><b>Lets see if the county of origin and shipping company name has any impact on purchase</b></p1>

In [ ]:
a=pd.get_dummies(sales['origin_country'])
a.info()

In [ ]:
sales1=sales.merge(a,how='outer',left_index=True, right_index=True)
sales1.info()

In [ ]:
to_be_used1=['AT','CN','GB','SG','US','VE','units_sold']
f1=plt.figure(figsize=(10,15))
plt.matshow(sales1[to_be_used1].corr(),fignum=f1.number)
plt.xticks(range(sales1[to_be_used1].shape[1]), sales1[to_be_used1].columns, fontsize=14, rotation=45)
plt.yticks(range(sales1[to_be_used1].shape[1]), sales1[to_be_used1].columns, fontsize=14)
cb=plt.colorbar()
cb.ax.tick_params(labelsize=14)

<b>From the matrix, we see that the country of orgin does not have significant impact on purchases</b>

<b>Lets see if the shipping option has any impact on the sales</b>

In [ ]:
b=pd.get_dummies(sales['shipping_option_name'])
b

In [ ]:
sales2=sales.merge(b,how='outer',left_index=True,right_index=True)
sales2.info()

In [ ]:
to_be_used2=list(sales2.columns[36:51])

In [ ]:
f2=plt.figure(figsize=(10,10))
plt.matshow(sales2[to_be_used2].corr(),fignum=f2.number)
plt.xticks(range(sales2[to_be_used2].shape[1]), sales2[to_be_used2].columns, fontsize=14, rotation=45)
plt.yticks(range(sales2[to_be_used2].shape[1]), sales2[to_be_used2].columns, fontsize=14)
cb=plt.colorbar()
cb.ax.tick_params(labelsize=14)

In [ ]:
#to_be_removed2.remove('units_sold')
for i in to_be_used2:
    if abs(sales2['units_sold'].corr(sales2[i]))>=0.5:
        print(i)

<p1><b>Lets assign each of the entry to a class based on their sales</b></p1>

In [ ]:
sales['units_sold'].describe()

<b><p1>We will use the following convention:</p1>
<ol>
<li>Sales value from 0 to 100: Very Cold</li>
<li>Sales value from 101 to 1000: Cold</li>
<li>Sales value from 1001 to 5000: Warm</li>
<li>Sales value from 5001 to 100000: Hot</li>    
</ol>

</b>

In [ ]:
category=[]
for i in sales['units_sold']:
    if i>=0 and i<=100:
        category.append('very_cold')
    elif i>=101 and i<=1000:
        category.append('cold')
    elif i>=1001 and i<=5000:
        category.append('warm')
    elif i>=5001:
        category.append('hot')
len(category)        

In [ ]:
sales['class']=category

In [ ]:
sales.head()

In [ ]:
l.append('units_sold')
l.append('class')

In [ ]:
l=np.array(l)
l

In [ ]:
sales_df=sales[l]
sales_df.head()

In [ ]:
sales_df.info()

In [ ]:
sales_df.dropna(inplace=True)

In [ ]:
sales_df.info()

To use scikit-learn library, we have to convert the Pandas data frame to a Numpy array:

In [ ]:
X=sales_df[['rating_count', 'rating_five_count', 'rating_four_count','rating_three_count', 'rating_two_count', 'rating_one_count','badge_product_quality', 'merchant_rating_count', 'units_sold']].values
X[0:5]

There features are all not in the same range. Hence lets bring them to the same order:

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

In [ ]:
y = sales_df['class'].values
y[0:5]

### Train Test Split  
Out of Sample Accuracy is the percentage of correct predictions that the model makes on data that that the model has NOT been trained on. Doing a train and test on the same dataset will most likely have low out-of-sample accuracy, due to the likelihood of being over-fit.

It is important that our models have a high, out-of-sample accuracy, because the purpose of any model, of course, is to make correct predictions on unknown data. So how can we improve out-of-sample accuracy? One way is to use an evaluation approach called Train/Test Split.
Train/Test Split involves splitting the dataset into training and testing sets respectively, which are mutually exclusive. After which, you train with the training set and test with the testing set. 

This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the data. It is more realistic for real world problems.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<h3>K nearest neighbor (KNN)</h3>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

### Training

Lets start the algorithm with k=4 for now:

In [ ]:
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

### Accuracy evaluation
In multilabel classification, __accuracy classification score__ is a function that computes subset accuracy. This function is equal to the jaccard_similarity_score function. Essentially, it calculates how closely the actual labels and predicted labels are matched in the test set.

In [ ]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

#### What about other K?
K in KNN, is the number of nearest neighbors to examine. It is supposed to be specified by the User. So, how can we choose right value for K?
The general solution is to reserve a part of your data for testing the accuracy of the model. Then chose k =1, use the training part for modeling, and calculate the accuracy of prediction using all samples in your test set. Repeat this process, increasing the k, and see which k is the best for your model.

We can calculate the accuracy of KNN for different Ks.

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

<h1>Thank You!!!<h1>

<h2>Feel free to share your comments on this work. If there are any errors, inaccuracies, or if you wish to share an alternative approach to tackle this problem, feel free to share them in the comments. I am here to learn so I am all ears to any constructive feedbacks :)</h2>

In [ ]:
pwd
